In [16]:
import twitter
import csv
import json
import re
import requests
import lxml.html as html
from urllib2 import urlopen
import pandas as pd
from datetime import datetime, timedelta
from dateutil import tz
import time


In [173]:
class TJLoader:

    def __init__(self):
        self._news_pages = "https://tjournal.ru/paper/page/{}"
        self._month_map = {u"января":"01", u"февраля":"02", u"марта":"03", u"апреля":"04", u"мая":"05", u"июня":"06", u"июля":"07", u"августа":"08", u"сентября":"09", u"октября":"10", u"ноября":"11", u"декабря":"12"}

    def get_news_uri(self, min_index=10, count=30):
        """

        :param min_index: int, индекс страницы, с которой нужно начать поиск
        :param count: int, количество страниц, которые нужно скачать
        :return: list. список ссылок на новости
        """
        links = []
        for i in range(count):
            page = html.parse(urlopen(self._news_pages.format(i+min_index)))
            divs = page.getroot().find_class('b-articles__b__title')

            for div in divs:
                links.append(div.getchildren()[1].get("href"))

        return links

    
    def _parse_date(self, date):
        date = date.replace(",", "")
        date = date.split(" ")
        
        converted_date = date[2]
        converted_date +="-"+self._month_map[date[1]]
        converted_date +="-"+date[0]
        
        converted_date +=" "+date[3]
        
        return converted_date
        

    def get_link_info(self, link):
        """

        :param link: str, url страницы с tjournal, для которой нужно собрать информацию
        :return: dict с данными со страницы
        """
        page = html.parse(urlopen(link))
        root = page.getroot()

        # заголовок
        title = root.find_class("b-article__title")
        title = title[0].find("h1").text

        # парсим количество просмотров
        view = root.get_element_by_id("hitsCount").text
        view = view.replace(" ", "")
        view = int(view)

        # Количество комментариев
        comments = root.find_class("b-article__infoline__comments")
        comment = int(comments[0].find("b").text.replace(" ", ""))

        # Теги
        tags = root.find_class("b-article__tags__tag")
        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        # Дата
        date = root.find_class("b-article__infoline__date")
        date = self._parse_date(date[0].text)
        

        return {"title": title, "views": view, "comments": comment, "tags": tag_list, "date": date}


    def get_tj_news_info(self, min_index=1, count=30, last_news=""):
        """
        :param min_index: int, индекс минимальной страницы, откуда начинаем поиск
        :param count: int, количество страниц, по которым ищем
        :last_news: str, id последней новости, которую нужно скачать
        :return: dict с данными со страницы
        """
        links = self.get_news_uri(min_index=min_index, count=count)
        link_info_list = []
        i = 0
        for link in links:
            link_info = self.get_link_info(link)
            link_id = link.split("/")[-1]
            
            # Если добрались до последней новости, которую хотим сохранить
            if (link_id == last_news):
                break
            link_info["id"] = link_id
            link_info_list.append(link_info)
            
            i+=1
            if i%10 == 0:
                print "Скачали ", i, " страниц"

        return link_info_list

In [204]:
class TwitterLoader:


    def __init__(self):
        CONSUMER_KEY = 'BOuuaMDhNhm6yx0rzqK8bMsbI'
        CONSUMER_SECRET = '3DybJwlkXd2vU6R385yLA8yJblYJltLtwojySD9AVs04ShauZ0'

        ACCESS_TOKEN_KEY = '3712177576-of3jzZ8gNmlPDfPjPyR0Ljw1Ao2IXdTqX9dZGDZ'
        ACCESS_TOKEN_SECRET = 'Ky7iKwByHNXX3UMfuMhv6UgVx2IhjLo3KmwpsBQz35wtG'

        self.api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET)
        self._month_dict = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "May":"5", "Jun":"6", "Jul":"7", "Aug":"8", "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}

        self._UTC_TIME_ZONE = tz.gettz('Europe/London')
        self._MOSCOW_TIME_ZONE = tz.gettz('Europe/Moscow')
        self._RATE_LIMIT = "[{u'message': u'Rate limit exceeded', u'code': 88}]"

        self._news_uri = "https://tjournal.ru/p/"



    def _parse_date(self, date):

        """

        :param date: str, дата в формате твиттера - "Sat Nov 21 17:00:29 +0000 2015"
        :return: str, дата в человеческом, но буржуйском формате, да еще и в Московском часовом поясе
        """
        date_array = date.split(' ')
        month = self._month_dict[date_array[1]]
        day = int(date_array[2])
        time = date_array[3]
        year = date_array[5]

        date = str(year)+"-"+str(month)+"-"+str(day)+" "+time
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        utc_date = date.replace(tzinfo=self._UTC_TIME_ZONE)
        moscow_date = utc_date.astimezone(self._MOSCOW_TIME_ZONE)

        return str(moscow_date).split("+")[0]



    def loadTweetWithLink(self, link, date):
        """

        :param link: str, ключевое слово для поиска, вданном случае - ссылка на письмо
        :param date: str, дата, до которой искать твиты
        :return: список словариков с информацией о УНИКАЛЬНЫХ твитах по запросу link
        """
        set_id = set()
        tweet_list = []
        result = self.api.GetSearch(term=link, until=date, count=100000)

        for res in result:

            tw_id = res.GetId()
            # Если мы уже обрабатывали этот твит - идем дальше
            if tw_id in set_id:
                continue

            retweeted_status = res.GetRetweeted_status()
            # Если это не ретвит
            if retweeted_status is None:
                is_retweet = 0
                retweeted_count = res.GetRetweetCount()
                favorite_count = res.GetFavoriteCount()
            else:
                is_retweet = 1
                retweeted_count = 0
                favorite_count = 0
            set_id.add(tw_id)

            created_at = res.GetCreatedAt()
            created_at = self._parse_date(created_at)

            # Данные о пользователе
            user = res.GetUser()
            followers_count = user.followers_count
            listed_count = user.listed_count
            friends_count = user.friends_count
            favourites_count = user.favourites_count
            statuses_count = user.statuses_count


            link = link.split("/")[-1]

            tw_dict= {
                    "id": link,
                    "tw_id":tw_id,
                    "retweeted_count": retweeted_count,
                    "favorite_count":favorite_count,
                    "is_retweet": is_retweet,
                    "created_at":created_at,
                    "user_followers_count":followers_count,
                    "user_listed_count": listed_count,
                    "user_friends_count":friends_count,
                    "user_favourites_count":favourites_count,
                    "user_statuses_count":statuses_count
                    }

            tweet_list.append(tw_dict)

        return tweet_list

    def _get_next_date(self, date, days):
        """

        :param date: str, дата
        :param days: int, количество дней
        :return: возвращает дату через days-дней после date
        """
        date = datetime.strptime(date, '%Y-%m-%d %H:%M')
        date += timedelta(days=days)
        return str(date).split(' ')[0]


    def load_tweets_by_term(self, news_list, days_after_news=2):
        """

        :param news_list: list(str), список словариков с информацией о новости
        :param days_after_news:  int, количество дней после публикации новости, до которой искать
        """
        result_list = []
        i = 0
        for news in news_list:
            tweets = []
            try:
                uri = self._news_uri + news["id"]
                until_date = self._get_next_date(news["date"], days_after_news)
                tweets = self.loadTweetWithLink(uri, until_date)
            except twitter.error.TwitterError as ex:
                print str(ex)
                if str(ex) == self._RATE_LIMIT:
                    sleep_time = self.api.GetSleepTime("search/tweets.json") #??? Почему-то не работает
                    print "Спим {} сек.".format(900)
                    time.sleep(900+2)
                    tweets = self.loadTweetWithLink(uri, until_date)

            result_list+= tweets
            
            i+=1
            if i%10 == 0:
                print "Собрано информация о", i, " новостях"
            
        return result_list

In [175]:
loader = TJLoader()
pages = loader.get_tj_news_info(min_index=1, count=4, last_news="android-chrome-pacsec")

Скачали  10  страниц
Скачали  20  страниц
Скачали  30  страниц
Скачали  40  страниц
Скачали  50  страниц
Скачали  60  страниц


In [177]:
news_info = pd.DataFrame(pages)
news_info.to_csv(OUT_NEWS_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [178]:
tw = TwitterLoader()
tweeter_data = tw.load_tweets_by_term(pages)

Собрано информация о 10  новостях
Собрано информация о 20  новостях
Собрано информация о 30  новостях
Собрано информация о 40  новостях
Собрано информация о 50  новостях
Собрано информация о 60  новостях


In [206]:
tweeter_data = pd.DataFrame(tweeter_data)
tweeter_data.to_csv(OUT_TWITTER_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

# После первого запуска, через несколько дней, запускать только часть, которая находится под этим заголовком!

In [221]:
OUT_NEWS_FILE = "news.csv"
OUT_TWITTER_FILE = "twitter.csv"

In [222]:
# читаем данные
news_df = pd.read_csv(OUT_NEWS_FILE, sep=",")
twitter_df = pd.read_csv(OUT_TWITTER_FILE, sep=",")

In [227]:
# находим самую "молодую новость"
news_df = news_df.sort_values(by=["date"], ascending=False)
last_news = news_df[news_df.index==0]["id"][0]

In [228]:
# скачивае информацию о новостях
loader = TJLoader()
pages = loader.get_tj_news_info(min_index=1, count=6, last_news=last_news)

In [229]:
len(pages)

0

In [ ]:
# Сохраняем в файлик
news_df = news_df.append(pd.DataFrame(pages))
news_df.to_csv(OUT_NEWS_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# Скачиваем данные из твиттера
tw = TwitterLoader()
tweeter_data = tw.load_tweets_by_term(pages)

In [ ]:
len(tweeter_data)

In [ ]:
# Сохраняем в файлик
twitter_df = twitter_df.append(pd.DataFrame(tweeter_data))
twitter_df.to_csv(OUT_TWITTER_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)